In [6]:
import os
from keras.preprocessing import image
import cv2

In [7]:
categories=['without_mask','with_mask']

In [8]:
data=[]
for category in categories:
    path=os.path.join('Dataset',category)
    label=categories.index(category)
    for file in os.listdir(path):
       img_path=os.path.join(path,file)
       img=cv2.imread(img_path)
       img=cv2.resize(img,(224,224))
       data.append([img,label])
    
       

In [9]:
len(data)

1965

In [10]:
import random

In [11]:
random.shuffle(data)

In [12]:
x=[]
y=[]
for features,label in data:
    x.append(features)
    y.append(label)    

In [13]:
import numpy as np

In [14]:
x=np.array(x)
y=np.array(y)

In [15]:
x.shape

(1965, 224, 224, 3)

In [16]:
y.shape

(1965,)

In [17]:
x=x/255

In [18]:
x[0]

array([[[0.79607843, 0.65490196, 0.61568627],
        [0.79607843, 0.65490196, 0.61568627],
        [0.79607843, 0.65490196, 0.61568627],
        ...,
        [0.93333333, 0.79607843, 0.74117647],
        [0.93333333, 0.79607843, 0.74117647],
        [0.93333333, 0.79607843, 0.74117647]],

       [[0.79607843, 0.65490196, 0.61568627],
        [0.79607843, 0.65490196, 0.61568627],
        [0.79607843, 0.65490196, 0.61568627],
        ...,
        [0.93333333, 0.79607843, 0.74117647],
        [0.93333333, 0.79607843, 0.74117647],
        [0.93333333, 0.79607843, 0.74117647]],

       [[0.79607843, 0.65490196, 0.61568627],
        [0.79607843, 0.65490196, 0.61568627],
        [0.79607843, 0.65490196, 0.61568627],
        ...,
        [0.93333333, 0.79607843, 0.74117647],
        [0.93333333, 0.79607843, 0.74117647],
        [0.93333333, 0.79607843, 0.74117647]],

       ...,

       [[0.09411765, 0.07843137, 0.09803922],
        [0.08627451, 0.07058824, 0.09019608],
        [0.07843137, 0

In [19]:
from sklearn.model_selection  import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2)

In [20]:
x_train.shape

(1572, 224, 224, 3)

In [21]:
x_test.shape

(393, 224, 224, 3)

In [22]:
from tensorflow.keras.applications.vgg16 import VGG16

In [ ]:
vgg=VGG16()

In [ ]:
vgg.summary()

In [ ]:
from keras import Sequential

In [ ]:
model=Sequential()

In [ ]:
for layer in vgg.layers[:-1]:
    model.add(layer)

In [ ]:
model.summary()

In [ ]:
for layer in model.layers:
    layer.trainable=False

In [ ]:
model.summary()

In [ ]:
from keras.layers import Dense

In [ ]:
model.add(Dense(1,activation='sigmoid'))

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
model.fit(x_train,y_train,epochs=5,validation_data=(x_test,y_test))

In [ ]:
cap=cv2.VideoCapture(0)

In [ ]:
def detect_face_mask(img):
    y_pred=model.predict_classes(img.reshape(1,224,224,3))
    return y_pred[0][0]

In [ ]:
def draw_label(img,text,pos,bg_color):
    text_size=cv2.getTextSize(text,cv2.FONT_HERSHEY_SIMPLEX,1,cv2.FILLED)
    end_x=pos[0]+text_size[0][0]+2
    end_y=pos[1]+text_size[0][1]-2
    cv2.rectangle(img,pos,(end_x,end_y),bg_color,cv2.FILLED)
    cv2.putText(img,text,pos,cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),1,cv2.LINE_AA)
    

In [ ]:
haar=cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

In [ ]:
def detect_face(img):
    coord=haar.detectMultiScale(img)
    return coord

In [ ]:
while True:
    ret,frame=cap.read()
    img=cv2.resize(frame,(224,224))
    coord=detect_face(cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY))
    for x,y,w,h in coord:
        cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),2)
    y_pred=detect_face_mask(img)
    if y_pred==0:
        draw_label(frame,"No mask",(30,30),(0,0,255))
    else:
        draw_label(frame,"mask",(30,30),(0,255,0))
    cv2.imshow('window',frame)
    if cv2.waitKey(1) & 0xFF== ord('x'):
        break
cap.release()
cv2.destroyAllWindows()

In [ ]:
sample=cv2.imread('"C:\Users\User\OneDrive\Desktop\Final project\Dataset\with_mask\img000001-with-mask.jpg"')
cv2.resize(sample,(224,224))

In [ ]:
detect_face_mask(sample)